# $S_3$ cross checks

In [ ]:
from luescher_nd.zeta.zeta3d import Zeta3D
from luescher_nd.zeta.extern.pyzeta import zeta

import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

import pandas as pd

In [ ]:
%load_ext blackcellmagic

In [ ]:
cut = 30

data = []

x = np.linspace(-4, 10, 2000)

for n_lambda in [10, 40, 80, 100]:
    z1 = Zeta3D(n_lambda, spherical=True)
    z2 = zeta
    z3 = Zeta3D(n_lambda, spherical=False)

    for key, z in {"$\Sigma_S$": z1, "$\int_S$": z2, "$\Sigma_C$": z3}.items():
        y = z(x)
        for xi, yi in zip(x, y):
            data.append({"type": key, "Lambda'": n_lambda, "y": yi, "x": xi})
            
df = pd.DataFrame(data)

df["y_norm"] = df.apply(
    lambda row: (row["y"] if row["y"] < cut else np.inf)
    if row["y"] > -cut
    else -np.inf,
    axis=1,
)

df.head()

In [ ]:
grid = sns.FacetGrid(data=df, col="type", hue="Lambda'", legend_out=True)

grid.map(plt.plot, "x", "y_norm")

grid.fig.set_dpi(200)
grid.fig.set_figheight(2)
grid.add_legend(title="$\Lambda'$")
grid.set_ylabels("$S_3(x; \Lambda')$")

plt.show(grid.fig)

In [ ]:
tf = df.set_index(["type", "Lambda'", "x"]).sort_index()[["y"]]

tf = tf.unstack(level=0)
tf.columns = [el[1] for el in tf.columns]

tf["$\Sigma_S - \int_S$"] = tf["$\Sigma_S$"] - tf["$\int_S$"]
tf["$\Sigma_C - \int_S$"] = tf["$\Sigma_C$"] - tf["$\int_S$"]
tf["$\Sigma_C - \Sigma_S$"] = tf["$\Sigma_C$"] - tf["$\Sigma_S$"]

tf = tf.drop(columns=["$\Sigma_S$", "$\Sigma_C$", "$\int_S$"])
tf = tf.stack().reset_index().rename(columns={"level_2": "type", 0: r"$\Delta S_3(x;\Lambda')$"})

tf.head()

In [ ]:
grid = sns.FacetGrid(data=tf, col="type", hue="Lambda'", legend_out=True)

grid.map(plt.plot, "x", r"$\Delta S_3(x;\Lambda')$")

grid.fig.set_dpi(200)
grid.fig.set_figheight(2)
grid.add_legend(title="$\Lambda'$")

plt.subplots_adjust()

plt.show(grid.fig)

In [ ]:
grid = sns.FacetGrid(
    data=tf.query("x == -4.000 and type != '$\Sigma_C - \Sigma_S$'"),
    hue="type",
    legend_out=True,
)

grid.map(plt.plot, "Lambda'", r"$\Delta S_3(x;\Lambda')$", marker=".", ls=":")

grid.fig.set_dpi(200)
grid.fig.set_figheight(2)
grid.add_legend(title="Type")

for ax in grid.axes.flatten():
    ax.set_yscale("log")
    ax.set_xscale("log")

plt.subplots_adjust()

plt.show(grid.fig)


In [ ]:
grid = sns.FacetGrid(
    data=tf.query("x == -4.000 and type == '$\Sigma_C - \Sigma_S$'"),
    hue="type",
    legend_out=True,
)

grid.map(plt.plot, "Lambda'", r"$\Delta S_3(x;\Lambda')$", marker=".", ls=":")

grid.fig.set_dpi(200)
grid.fig.set_figheight(2)
grid.add_legend(title="Type")

for ax in grid.axes.flatten():
    #ax.set_yscale("symlog")
    ax.set_xscale("log")

plt.subplots_adjust()

plt.show(grid.fig)

